# Website Demo for Infection Risk Calculator with Information Retrieved from Univeristy Systems

**This is the demo for our website application which estimates infection risk based on building information** 

Import necessary packages and files

In [1]:
from jupyter_dash import JupyterDash
import dash


import os

import sys
import dash_core_components as dcc
import dash_html_components as html
import dash
import sys
import os
sys_path = os.path.dirname(os.getcwd())
src_path = sys_path + "/src"

sys.path.insert(1, src_path)
from calculator import *
assumptions = var
src_path = sys_path + "/notebook"

sys.path.insert(1, src_path)

In [3]:
pip install jupyter-dash # Install jupyter-dash for running HTML in Python

  Using cached jupyter_dash-0.4.0-py3-none-any.whl (20 kB)
  Using cached ansi2html-1.6.0-py3-none-any.whl (14 kB)
  Using cached dash-1.19.0.tar.gz (75 kB)
  Using cached retrying-1.3.3.tar.gz (10 kB)
  Using cached Flask_Compress-1.8.0-py3-none-any.whl (7.2 kB)
  Using cached plotly-4.14.3-py2.py3-none-any.whl (13.2 MB)
  Using cached dash_renderer-1.9.0.tar.gz (1.0 MB)
  Using cached dash_core_components-1.15.0.tar.gz (3.5 MB)
  Using cached dash_html_components-1.1.2.tar.gz (188 kB)
  Using cached dash_table-4.11.2.tar.gz (1.8 MB)
  Created wheel for dash: filename=dash-1.19.0-py3-none-any.whl size=84016 sha256=6be76a0639d2b39a91c31ed8e10714545063ee955b4ceddfff09b7a434cd5ba1
  Stored in directory: c:\users\hinanawits\appdata\local\pip\cache\wheels\85\e9\f4\033d9bd19fe04ba8b3b41b6cfaec3c4634e85bdcdd82858ff3
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11435 sha256=d39941020b7b12c1b8db7b31100f638811b12ef057137bcec51242e73c54b1d0
  Stored in directory: c

Import data

In [2]:
rid_path = 'rm.csv'
sys_path = os.path.dirname(os.getcwd())


data_path = sys_path + "/data/"

room_df = pd.read_csv(data_path + rid_path)
rooms = room_df.copy()

rooms_id = []
for rid in room_df['Room']:
    rooms_id.append({'label': rid, 'value': rid})
    
rooms.head()

,Room,Area,Height,Windows,Capacity,C19_Occupancy,VAVmin,VAVmax
0,CENTER 101,3428,21.0,NaN,295.0,50.0,3500.0,7000.0
1,CENTER 115,2143,21.0,NaN,196.0,31.0,1290.0,4300.0
2,CENTER 119,2144,21.0,NaN,196.0,30.0,1290.0,4300.0
3,GH 242,2956,NaN,NaN,417.0,50.0,935.0,2325.0
4,LEDDN AUD,1888,NaN,NaN,216.0,30.0,4000.0,12000.0


demo run for debug purpose. 

In [5]:
ui_calc("Lecture", "CENTER 101", 110, 100, data_path + rid_path, "max")

The resulting risk of infection is 1.293014369281087%
It is predicted that 0.01293014369281087 x 100 = 1 susceptible occupants will be infected
The resulting risk of infection is 21.448979453682725%
It is predicted that 0.21448979453682726 x 100 = 21 susceptible occupants will be infected


0.11370996911481906

**Build Website** 

In [7]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

server = app.server
activities = [{'label':'Lecture', 'value':'Lecture'}, {'label':'Studying', 'value':'Studying'}, {'label':'Singing', 'value':'Singing'}, {'label':'Social', 'value':'Social'}, {'label':'Exercising', 'value':'Exercising'}]
app.layout = html.Div([
    html.H6("Event Information"),
    #MAKE ROOM ID A DROP DOWN?
    html.Div(["RoomID: ",
              dcc.Dropdown(id='room-dropdown', value='test', options = rooms_id)]),
    html.Br(),
    html.Div(["Duration of Event (min): ",
              dcc.Input(id='time-input', value = 0, type='number')]),
    html.Br(),
    html.Div(["Number of Occupants: ",
              dcc.Input(id='occupant-input', value = 0, type='number')]),
    html.Br(),
    html.Div(["Activity: ",
              dcc.Dropdown(id='activity-dropdown', value ='test', options=activities)]),
#     html.Br(),
#     html.Button('Reset', id='reset-button'),
    html.Br(),
    html.Button('Go', id = 'go-button', n_clicks = 0),
    html.Br(),
    html.Div(id = 'calc-output', children = 'Enter values to calculate risk')

])

@app.callback(
    dash.dependencies.Output('calc-output', 'children'),
    [dash.dependencies.Input('go-button', 'n_clicks')],
    [dash.dependencies.Input('activity-dropdown', 'value')],
    [dash.dependencies.Input('room-dropdown', 'value')],
    [dash.dependencies.State('time-input', 'value')],
    [dash.dependencies.State('occupant-input', 'value')]
)
def update_calc(n_clicks, activity_dropdown, room_input, time_input, occupant_input):
    if n_clicks >= 1:
        comp_ir = ui_calc(activity_dropdown, room_input, time_input, occupant_input, data_path + rid_path)
        total_inf = int(occupant_input * comp_ir)
        to_return = 'The risk of an individual infected because of holding a(n) {} event for {} minutes in {} is {}%, given the most recent infection rates. With {} occupants, it is likely that {} occupant(s) will be infected.'.format(activity_dropdown, 
                                                                                                                                time_input, 
                                                                                                                                room_input, 
                                                                                                                                round((comp_ir * 100),2), 
                                                                                                                                occupant_input,
                                                                                                                                total_inf)
        return to_return
    else:
        return 'Enter Values to get risk calculation'

In [8]:
app.run_server(mode="inline")

The resulting risk of infection is 1.293014369281087%
It is predicted that 0.01293014369281087 x 100 = 1 susceptible occupants will be infected
The resulting risk of infection is 21.448979453682725%
It is predicted that 0.21448979453682726 x 100 = 21 susceptible occupants will be infected
